Documentação IBGE Geral 
https://servicodados.ibge.gov.br/api/v1/localidades

Documentação Distritos - Obtém o conjunto de distritos do Brasil
https://servicodados.ibge.gov.br/api/v1/localidades#api-Distritos-distritosGet

Guia Request 
https://requests.readthedocs.io/projects/pt/pt-br/latest/user/quickstart.html#respota-json

In [0]:
%python
%pip install requests

In [0]:
%python
# inportando dados da API IBGE
import requests
import json 

url = 'https://servicodados.ibge.gov.br/api/v1/localidades/distritos' # origem dos dados api 
response = requests.get(url)
data = response.json()
data




**Campos aninhados em JSON** 
<br>
Referem-se a dados que estão organizados em uma estrutura hierárquica, onde um campo pode conter outros campos dentro de si. Essa estrutura cria um relacionamento em camadas ou níveis de dados, permitindo representar objetos complexos de forma mais organizada e legível.Esse tipo de estrutura é comum em JSON ou em bancos de dados NoSQL

Salvando em arquivo no seu ambiente caso queira (Obs: deixei tambem no material para Download)

In [0]:
%python
import requests
import json

# Obter os dados JSON da URL
url = "https://servicodados.ibge.gov.br/api/v1/localidades/distritos"
response = requests.get(url)
data = response.json()

# Caminho destino Worckspace
path = "/Workspace/Users/edmilson.onanalytics@gmail.com/Databricks-Curso/00.Arquivos-Datasets/Dados IBGE/distritos_IBGE.json"

# Salvar o JSON 
with open(path, 'w', encoding='utf-8') as f:
    json.dump(data, f, ensure_ascii=False, indent=4)

In [0]:
import requests
import json

# Obter os dados JSON da URL
url = "https://servicodados.ibge.gov.br/api/v1/localidades/distritos"
response = requests.get(url)
data = response.json()

# Caminho destino dbfs
path = "/dbfs/FileStore/tables/distritos_IBGE.json"

# Salvar 
with open(path, "w", encoding="utf-8") as f:
    json.dump(data, f, ensure_ascii=False, indent=4)



Ducumentação de referencia 

[struct-type](https://learn.microsoft.com/fi-fi/azure/databricks/sql/language-manual/data-types/struct-type)
<br>
[struct-type- Spark](https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/api/pyspark.sql.types.StructType.html)


In [0]:
%python
#Transformar resposta da Api em uma tabela 

import requests
import json 
from pyspark.sql.types import StructType, StructField, StringType, IntegerType

url = 'https://servicodados.ibge.gov.br/api/v1/localidades/distritos' # origem dos dados api 
response = requests.get(url)
data = response.json()
data

In [0]:

schema = StructType([
    StructField("id", IntegerType(), True),
    StructField("nome", StringType(), True),
    StructField("municipio", StructType([
        StructField("id", IntegerType(), True),
        StructField("nome", StringType(), True),
        StructField("microrregiao", StructType([
            StructField("id", IntegerType(), True),
            StructField("nome", StringType(), True),
            StructField("mesorregiao", StructType([
                StructField("id", IntegerType(), True),
                StructField("nome", StringType(), True),
                StructField("UF", StructType([
                    StructField("id", IntegerType(), True),
                    StructField("sigla", StringType(), True),
                    StructField("nome", StringType(), True),
                    StructField("regiao", StructType([
                        StructField("id", IntegerType(), True),
                        StructField("sigla", StringType(), True),
                        StructField("nome", StringType(), True)
                    ]), True)
                ]), True)
            ]), True)
        ]), True)
    ]), True)
])


df = spark.createDataFrame(data , schema=schema)

In [0]:
# ver dataframe
display(df)

In [0]:
# salvar Df no formato de tabela no banco de dados
df.write.mode("overwrite").saveAsTable("distritos")

In [0]:
%sql
show tables;

In [0]:
%sql
select * from distritos

2° Opção se nao conseguir ler em API: Fazer leitura no arquivo da pasta para download

A opção spark.read.option("multiline", "true") no Apache Spark é usada quando você está lendo arquivos JSON que possuem múltiplas linhas ou que têm um formato em que cada objeto JSON pode ocupar mais de uma linha. Ao definir esta opção como true, você está instruindo o Spark a ler o arquivo como um arquivo JSON multiline, ou seja, um arquivo JSON que possui um único objeto JSON por linha, mas com possíveis quebras de linha dentro de cada objeto.

Exemplo de uso 
df = spark.read.option("multiline", "true").json("arquivo.json")


In [0]:
#Salvando o arquivo JSON como um DataFrame em banco de dados com a função read.option
path='file:/Workspace/Users/edmilson.onanalytics@gmail.com/Databricks-Curso/00.Arquivos-Datasets/Dados IBGE/distritos_IBGE.json'
df_arquivo = spark.read.option("multiLine", True).json(path)
display(df_arquivo)# ver os dados do arquivo json 




In [0]:
# salvar Df no formato de tabela no banco de dados
df_arquivo.write.mode("overwrite").saveAsTable("distritos_arquivo")

In [0]:
%sql
show tables

In [0]:
%sql
select * from distritos

In [0]:
%sql
select 
  id
  ,nome as nome_municipio
  --,municipio
  ,municipio.microrregiao.nome as nome_microrregiao
  ,municipio.microrregiao.mesorregiao.UF.sigla  as UF_sigla
  ,municipio.microrregiao.mesorregiao.UF.nome  as UF_nome
  ,municipio.microrregiao.mesorregiao.UF.regiao.nome  as Regiao
 from distritos
--where id = 510517605

In [0]:
%sql
describe detail distritos

In [0]:
%sql
describe extended distritos

In [0]:
%sql
-- deletar tabelas caso queira no final 
drop table if exists distritos;
drop table if exists distritos_arquivo